# Le Pattern Matching

---

> À l'instar du `if let` qui surpasse les simples conditions, le pattern matching est superieur au **switch case** du C/C++. Le pattern matching peut travailler avec n'importe quel type de donnée, ensuite, tout comme le if let, il peut déstructurer les tuples, les structs etc... Et enfin, **le compilateur s'assure que toutes les possibilités de valeur ont bien été prises en compte**, c'est à mon goût le point le plus fort du pattern matching, cela évite beaucoup d'erreurs. Dans la pratique, l'on va donc éviter de mettre un défaut à la fin du match contrairement à ce que l'on ferait dans un vulgaire switch case.

## Introduction

- Le compilateur s'assure que toutes les possibilités ont été couvertes. Voyons un exemple :

In [6]:
enum Color {
    Red,
    Black,
}
let c = Color::Red;
match c {
    Color::Red => println!("It is RED"),
}

Error: non-exhaustive patterns: `Color::Black` not covered

> Le variant Black n'a pas été testé et le message d'erreur du compilateur nous en informe. Si a la place du match on aurait utilisé un switch case en C, le compilateur aurait laisse faire...

> Il existe alors deux solutions :
>> - Soit on code un **cas par défaut**
>> - Ou bien l'on s'efforce de **couvrir tous les variants** de l'énumération

### Un motif universel comme cas par défaut

- Dans le chapitre précédent, nous avons vu que nous n'avions pas besoin de spécifier toutes les données d'un motif et nous récupérions ainsi certaines données dans des variables. Mais que ce serait-il passe si tout le motif entier aurait été ignoré ?

In [8]:
// Pas de #[derive(PartialEq, Eq)]
enum Button {
    Up,
    Down,
    Middle,
}
// Tiens, un passage par reference ici
fn test_button(button: &Button) {
    if let b = button {
        println!("It is a button... sure...");
    }   
}
test_button(&Button::Up);
test_button(&Button::Down);
test_button(&Button::Middle);

It is a button... sure...
It is a button... sure...
It is a button... sure...


> Il n'y a ici aucune contrainte de motif ! Dans tous les cas, le if let est valide !

- On peut se servir de la même idée pour définir un cas par défaut dans le pattern matching

In [12]:
enum Color {
    Red,
    Black,
}
let c = Color::Black;
match c {
    Color::Red => println!("It is RED"),
    other_color => println!("I know it is not RED !"),
}

I know it is not RED !


()

> **NB** Il est important de mettre le cas par défaut en dernier.

In [15]:
enum Color {
    Red,
    Black,
}
let c = Color::Red;
match c {
    // Le cas par defaut est en premier. Meme si la couleur est rouge, on ne le saura pas !
    other_color => println!("I know it is not RED !"),
    Color::Red => println!("It is RED"),
}

I know it is not RED !


()

> Pourtant, la couleur était bien le rouge !

* On préfixe généralement les noms des variables par défaut qui ne nous intéresse pas par un **underscore** `_` voire par un underscore seul, ainsi la forme la plus courante sera :

In [17]:
enum Color {
    Red,
    Black,
}
let c = Color::Red;
match c {
    Color::Red => println!("It is RED"),
    _ => println!("I know it is not RED !"),
}

It is RED


()

- Tout comme avec if let, il est tout à fait possible d'utiliser **l'opérateur OR** et les **ranges** dans un match pattern :

In [2]:
let number = 13;
// TODO ^ Try different values for `number`

println!("Tell me about {}", number);
match number {
    // Match a single value
    1 => println!("One!"),
    // Match several values
    2 | 3 | 5 | 7 | 11 => println!("This is a prime"),
    // TODO ^ Try adding 13 to the list of prime values
    // Match an inclusive range
    13..=19 => println!("A teen"),
    // Handle the rest of cases
    _ => println!("Ain't special"),
    // TODO ^ Try commenting out this catch-all arm
}

Tell me about 13
A teen


()

> **NB** On retrouvera souvent en Rust des variables préfixées par l'underscore ou bien des underscore seuls pour tout ce qui parait inutile de lire.

### La couverture de tous les motifs possibles

* Pour l'énumération Color, couvrir tous les motifs semble être une bien meilleure solution :

In [19]:
enum Color {
    Red,
    Black,
}
let c = Color::Black;
match c {
    Color::Red => println!("It is RED"),
    Color::Black => println!("It is BLACK"),
}

It is BLACK


()

- Voici un autre exemple de couverture de tous les motifs. Notez la récupération de freq dans le premier motif ou l'utilisation de l'underscore solitaire dans le second :

In [20]:
enum Target {
    X86(u32),
    Ia32(u32),
    Mips,
}
use Target::*;
let my_target = Ia32(25);
match my_target {
    X86(freq) => println!("X86 at frequency {} mhz", freq),
    Ia32(_) => println!("Ia32 but we dont worried about frequency"), 
    Mips => println!("Just an other Mips again"), 
}

Ia32 but we dont worried about frequency


()

## Les différentes déstructurations

### Cas d'une structure

- Nous avons déjà croise la déstructuration d'un motif d'une structure dans le chapitre traitant de if let, déstructurer consiste a remplacer une partie de cette structure par une variable que nous récupérerons ou pas ou bien même a ignorer certaines valeurs dans le motif :

In [14]:
struct Point3D {
    i: usize,
    j: usize,
    k: usize,
}
fn test_3d_point(p: Point3D) {
    match p {
        // Pas de destructuration ici
        Point3D { i: 2, j: 5, k: 2} => println!("1: A point with i = 2, j = 5 and k = 2"),
        // Recuperation de la variable j
        Point3D { i: 2, j: second_value, k: third_value} => println!("2: j = {} k = {}", second_value, third_value),
        // La syntaxe .. implique que la suite du motif de la structure sera ignoree, ici j et k
        Point3D { i, ..} => println!("3: Others cases... i value is {}", i),
    }
}
test_3d_point(Point3D { i: 2, j: 5, k: 2});
test_3d_point(Point3D { i: 2, j: 5, k: 10});
test_3d_point(Point3D { i: 8, j: 10, k: 10});

1: A point with i = 2, j = 5 and k = 2
2: j = 5 k = 10
3: Others cases... i value is 8


> **NB** Il est très important de bien spécifier le nom des variables de la structure lors du match. Ainsi, je dois écrire `Point3D { i: 2, j: 5, k: 2} => ...` et non Point3D { 2, 5, 2} => ... ! L'ordre importe peu, lors du second test, j'aurai pu écrire `Point3D { j: second_value, k: third_value, i: 2} => ...` L'oubli du nom d'une variable pour une structure est une erreur que font fréquemment les débutants, car ils confondent avec un tuple.

### Cas d'une énumération

- La déstructuration d'une énumération consiste le plus souvent à tester la présence d'un variant et a récupérer les données associées a ce dernier :

In [30]:
enum MyColor {
    // Ces 3 premieres couleurs sont des variants simples.
    Red,
    Blue,
    Green,
    // Ces modeles sont des tuples
    RGB(u32, u32, u32),
    CMYK(u32, u32, u32, u32),
}

fn test_my_color(color: MyColor) {
    match color {
        MyColor::Red   => println!("1: The color is Red!"),
        MyColor::Blue  => println!("2: The color is Blue!"),
        MyColor::Green => println!("3: The color is Green!"),
        // La premiere valeur doit etre egale a 10.
        MyColor::RGB(10, g, b) =>
            println!("4: Red must be 10, green: {}, and blue: {}!", g, b),
        MyColor::RGB(r, g, b) =>
            println!("5: Red: {}, green: {}, and blue: {}!", r, g, b),
        //  Les 3 premiers chamos sont ignores.
        MyColor::CMYK(.., k) =>
            println!("6: The first three values are ignored, key (black): {}!", k),
        // Tous les cas ont ete couverts.
    }
}
test_my_color(MyColor::RGB(122, 17, 40));
test_my_color(MyColor::RGB(10, 17, 40));
test_my_color(MyColor::Red);
test_my_color(MyColor::CMYK(13, 20, 100, 42));

5: Red: 122, green: 17, and blue: 40!
4: Red must be 10, green: 17, and blue: 40!
1: The color is Red!
6: The first three values are ignored, key (black): 42!


### Cas d'un tableau

- La déstructuration d'un tableau obéit aux mêmes principes.

In [31]:
let triple = [0, -2, 3];
println!("Tell me about {triple:?}");
match triple {
    [0, y, z] => println!("First is 0, y = {y}, and z = {z}"),
    [1, ..]   => println!("First is 1 and the rest were ignored"),
    _         => println!("All elements were ignored"),
}

Tell me about [0, -2, 3]
First is 0, y = -2, and z = 3


()

- Je le découvre en écrivant ce cours :) Mais il semble tout à fait possible de récupérer un sous-tableau d'un tableau lors d'un match grâce au symbole **@** :

In [38]:
fn check_array(arr: [usize; 6]) {
    match arr {
        [2, tail @ ..] => println!("1: Begin wih 2, tail subarray is {:?}", tail),
        [3, middle @ .., 11] => println!("2: Begin with 3 ans end with 11, midle subarray is {:?}", middle),
        [begin @ .., 11, 13] => println!("3: Terminate with 11, 12, begin subarray {:?}", begin),
        [default @ ..] => println!("4: Default case: array is {:?}", default),
    }
}
check_array([2, 3, 5, 7, 11, 13]);
check_array([3, 4, 5, 6, 7, 11]);
check_array([0, 0, 0, 0, 11, 13]);
check_array([0, 12, 0, 0, 11, 98]);

1: Begin wih 2, tail subarray is [3, 5, 7, 11, 13]
2: Begin with 3 ans end with 11, midle subarray is [4, 5, 6, 7]
3: Terminate with 11, 12, begin subarray [0, 0, 0, 0]
4: Default case: array is [0, 12, 0, 0, 11, 98]


### Cas d'un tuple

- Rien de particulier, on suit les mêmes règles que précédemment :

In [21]:
let tuple: (u8, u8, u8, u8, u8) = (0, 1, 1, 2, 3);
// destructured tuple
match tuple {
    (0..=5, .., v5) => println!("v5 = {}", v5),
    (6..=u8::MAX, .., v4, v5) => println!("v4 = {} v5 = {}", v4, v5),
}

v5 = 3


()

## L'ajout d'expression booléennes

- Il est aussi possible d'ajouter l'évaluation d'expressions avec `if` au pattern matching. Elles peuvent constituer un filtre supplémentaire bien que ce dernier ne sera pas évalué par le compilateur afin de savoir si tous les cas ont été couverts, c'est une limitation actuelle de Rust et j'ignore si cela est corrigé un jour. Regardons cet exemple :

In [39]:
let v: u32 = 142;
match v {
    i if i < 100_u32 => println!("Below than 100: {}", i),
    i if i >= 100_u32 => println!("Above or equal than 100: {}", i),
}

Error: non-exhaustive patterns: `_` not covered

> Ici la variable **v** est un u2, une variable dont les valeurs seront forcément comprises entre 0 et u32::MAX. Tous les cas semblent donc couverts puisque dans la première ligne du match on demande si i est inférieur ou égal a 100 et dans le second, on couvre toutes les valeurs supérieures à 100. Pourtant, le compilateur n'est pas d'accord et nous retourne une erreur...

- Cela parait donc contre-intuitif et notre seul possibilité de résolution de l'erreur est d'ajouter un cas par défaut qui n'arrivera pourtant jamais :

In [45]:
// Range inclusive.
for v in 0..=u32::MAX {
    match v {
        i if i < 100_u32 => {},
        i if i >= 100_u32 => {},
        _default => panic!("Woot ?"), // Le programme paniquera si on tombe dans le default.
    }
}
println!("All is okay");

All is okay


- Pour terminer le chapitre, voici un exemple plus complet d'ajout d'expression booléenne :

In [50]:
enum Target {
    X86(u32),
    Ia32(u32),
    Mips,
}
fn test_my_target(my_target: Target) {
    use Target::*;
    match my_target {
        X86(freq) if freq < 30 => println!("1: X86 at LOW frequency {} mhz", freq), // Condition < 30
        X86(freq) if freq >= 30 => println!("2: X86 at HIGH frequency {} mhz", freq), // Condition >= 30
        Ia32(_) => println!("3: Ia32 but we dont worried about frequency"), 
        Mips => println!("4: Just an other Mips again"),
        _ => panic!("Woot ?!?"), // Dummy default case.
    }
}
test_my_target(Target::X86(15));
test_my_target(Target::X86(120));
test_my_target(Ia32(65));
test_my_target(Mips);

1: X86 at LOW frequency 15 mhz
2: X86 at HIGH frequency 120 mhz
3: Ia32 but we dont worried about frequency
4: Just an other Mips again


## Exercices

### Écrire une séquence de code avec match pour tous les cas :

In [51]:
enum MyOption {
    Some(usize),
    None,
}
let opt = MyOption::Some(11);

In [35]:
// Ecrire le code ici

```
match opt {
    MyOption::Some(value) => println!("It is Some({})", value),
    MyOption::None => println!("MyOption is empty !"),
}
```

### Écrire une séquence de code avec match pour tous les cas :

In [52]:
enum MyResult {
    Ok(usize),
    Err(&'static str),
}
let res = MyResult::Err("Unexpected error... Woot ?!?");

In [34]:
// Ecrire le code ici

```
match res {
    MyResult::Ok(value) => println!("It is Ok({})", value),
    MyResult::Err(s) => println!("The error is {}", s),
}
```